In [73]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data.shape

(20800, 5)

In [4]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
# we will using only 'title' column as independent 
# and dependent variable is label

In [6]:
new_data = data.copy()

In [7]:
# new_data.drop(columns=['author'],inplace=True)

In [8]:
new_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
new_data = new_data.dropna()

In [10]:
new_data.shape

(18285, 5)

In [11]:
new_data.reset_index(inplace=True)

In [12]:
new_data.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
import nltk
import re
from nltk.stem import WordNetLemmatizer  # lemmatozation
from nltk.corpus import stopwords

In [14]:
wnl = WordNetLemmatizer()

In [16]:
corpus_list = []
for i in range(len(new_data)):
    view = re.sub('[^a-zA-Z]',' ',new_data['title'][i])
    view = view.lower()
    view = view.split()
    view = [wnl.lemmatize(vi) for vi in view if vi not in stopwords.words('english')]
    view = ' '.join(view)
    corpus_list.append(view)

In [17]:
corpus_list[0]

'house dem aide even see comey letter jason chaffetz tweeted'

# here we use Bag_Of_Wrods

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

- `ngram_range` take min and max value, by default it will take min = 1 and max = 1
- For example an `ngram_range` of (1, 1) means only unigrams means only that words which is single 
- (1, 2) means unigrams and bigrams words which appear single as well as two words Ex: `uni( abl )` and  `bi(new york)`
- (2, 2)  means only bigrams.  and so on..


In [19]:
# ngram_range = (1,3) take combination of one word, two words and three words
cv = CountVectorizer(max_features=5000, ngram_range=(1,3))  # uni, bi and tri

In [20]:
X = cv.fit_transform(corpus_list).toarray()

In [21]:
X.shape

(18285, 5000)

In [ ]:
# cv.get_feature_names()[:20]  # only one word, two pair and three pair 

In [22]:
# see how independent varibles look like
word_df = pd.DataFrame(X,columns=cv.get_feature_names())

In [23]:
word_df.head()

,abandon,abandoned,abc,abc news,abe,abedin,able,abortion,abroad,absolutely,...,zealand,zero,zika,zika virus,zionist,zone,zone new,zone new york,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
y = new_data['label'] # dependent variable

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=0)

## 1. MultinomailNB Algo

In [27]:
from sklearn.naive_bayes import MultinomialNB

In [28]:
mnb = MultinomialNB()

In [29]:
mnb.fit(x_train,y_train)

MultinomialNB()

In [30]:
y_pred = mnb.predict(x_test)

In [53]:
from sklearn.metrics import accuracy_score , confusion_matrix, plot_confusion_matrix

In [32]:
print("Accuracy Score: ",accuracy_score(y_test,y_pred))

Accuracy Score:  0.8990156762668611


In [69]:
print("Confusion matrix: ")
confusion_matrix(y_test,y_pred)

Confusion matrix: 


array([[2770,  314],
       [ 240, 2162]], dtype=int64)

- for better and constant accuracy 
#### K-Fold Cross Validation

In [34]:
from sklearn.model_selection import cross_val_score

In [39]:
scores = cvs = cross_val_score(mnb, X,y, cv=5)

In [40]:
scores

array([0.90210555, 0.90566038, 0.90265245, 0.89608969, 0.89773038])

In [50]:
print("Final Accuracy Score: ",scores.mean())

Final Accuracy Score:  0.9008476893628657


## 2. MultinomailNB Algo with Hyperparameter

In [84]:
# mnb1 = MultinomialNB(alpha=0.1)
val = np.arange(0,1,0.1)

In [86]:
final_Score = 0
for i in val:
    mnb2 = MultinomialNB(alpha = i)
    mnb2.fit(x_train,y_train)
    y_pred1 = mnb2.predict(x_test)
    score = accuracy_score(y_test,y_pred1)
    print("Alpha: ",i,"Score: ",score)

c:\users\hp\appdata\local\programs\python\python38\lib\site-packages\sklearn\naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


Alpha:  0.0 Score:  0.8869850528618302
Alpha:  0.1 Score:  0.8977397010572365
Alpha:  0.2 Score:  0.8975574188844331
Alpha:  0.30000000000000004 Score:  0.8975574188844331
Alpha:  0.4 Score:  0.8977397010572365
Alpha:  0.5 Score:  0.8988333940940576
Alpha:  0.6000000000000001 Score:  0.8995625227852716
Alpha:  0.7000000000000001 Score:  0.8997448049580751
Alpha:  0.8 Score:  0.8995625227852716
Alpha:  0.9 Score:  0.8995625227852716


#### Checking Which words are Fake and Which one is Real

In [88]:
feature_name = cv.get_feature_names()

- If word have biggest negative value then that word is real word Ex: -5, -4, -3, -2,........

In [89]:
sorted(zip(mnb.coef_[0], feature_name),reverse=True)[:20]

[(-3.946246344405199, 'trump'),
 (-4.236706747677706, 'hillary'),
 (-4.32820860805858, 'clinton'),
 (-4.947496727480401, 'election'),
 (-5.097341695596448, 'new'),
 (-5.200199081036156, 'comment'),
 (-5.229799550812447, 'video'),
 (-5.352961835588566, 'war'),
 (-5.352961835588566, 'hillary clinton'),
 (-5.387533179653655, 'fbi'),
 (-5.493452198537392, 'email'),
 (-5.545039567327885, 'world'),
 (-5.5809715765539485, 'obama'),
 (-5.670202710281892, 'donald'),
 (-5.704104261957573, 'donald trump'),
 (-5.71799337411824, 'russia'),
 (-5.821025600016475, 'day'),
 (-5.821025600016475, 'america'),
 (-5.836650917919556, 'american'),
 (-5.944639555983652, 'vote')]

- If word have lowest negative value then the word is fakest word Ex: -10, -11, -12 .....

In [90]:
sorted(zip(mnb.coef_[0], feature_name))[:20]

[(-10.680838004378147, 'abe'),
 (-10.680838004378147, 'abroad'),
 (-10.680838004378147, 'abuse new'),
 (-10.680838004378147, 'abuse new york'),
 (-10.680838004378147, 'accord'),
 (-10.680838004378147, 'act new'),
 (-10.680838004378147, 'act new york'),
 (-10.680838004378147, 'advice'),
 (-10.680838004378147, 'adviser new'),
 (-10.680838004378147, 'adviser new york'),
 (-10.680838004378147, 'affordable'),
 (-10.680838004378147, 'age new'),
 (-10.680838004378147, 'age new york'),
 (-10.680838004378147, 'agenda breitbart'),
 (-10.680838004378147, 'ailes'),
 (-10.680838004378147, 'aleppo new'),
 (-10.680838004378147, 'aleppo new york'),
 (-10.680838004378147, 'ali'),
 (-10.680838004378147, 'america breitbart'),
 (-10.680838004378147, 'america new york')]

# Test the model By using Test Dataset

In [185]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [186]:
test_data.shape

(5200, 4)

In [187]:
test_data.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [98]:
# handel null value

In [188]:
test_data.fillna('mode',inplace=True)

In [189]:
test_data.shape

(5200, 4)

In [190]:
# we train model by title so we test by title
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [191]:
wnl = WordNetLemmatizer()

In [192]:
lemmat = []
for i in range(len(test_data)):
    view1 = re.sub('[^a-zA-Z]',' ',test_data['title'][i])
    view1 = view1.lower()
    view1 = view1.split()
    view1 = [wnl.lemmatize(vi) for vi in view1 if vi not in stopwords.words('english')]
    view1 = ' '.join(view1)
    lemmat.append(view1)

In [193]:
from sklearn.feature_extraction.text import CountVectorizer 

In [194]:
cv1 = CountVectorizer(max_features=5000, ngram_range=(1,3))

In [195]:
test = cv1.fit_transform(lemmat).toarray()

- while traing the the model we set `max_feature` as `5000` so the model get trained with `5000 features` so in test data we also set the `max_feature` as `5000`

In [196]:
x_test.shape , test.shape 

((5486, 5000), (5200, 5000))

## Prediction On Test_Data By First Model MultinomailNB Algo without Hyperparameter

In [197]:
final_pred = mnb.predict(test)